In [260]:
import pandas as pd
import re
import numpy as np
import torch
from tqdm import tqdm
from sklearn.decomposition import TruncatedSVD

In [261]:
data_ans = pd.read_csv("data/answer_metadata.csv")
#AnswerId             DateAnswered Confidence  GroupId  QuizId SchemeOfWorkId [1508917 rows x 6 columns]
data_ans = data_ans[['AnswerId','DateAnswered']]
#AnswerId             DateAnswered  [1508917 rows x 2 columns]


data_ques = pd.read_csv("data/question_metadata.csv")
#QuestionId         SubjectId   [948 rows x 2 columns]


data_stu = pd.read_csv("data/student_metadata.csv")
#UserId  Gender              DateOfBirth PremiumPupil   [6148 rows x 4 columns]
data_stu = data_stu[['UserId','Gender','DateOfBirth']]
#UserId  Gender              DateOfBirth    [6148 rows x 3 columns]



data_relation = pd.read_csv("data/train_task_4.csv")
#QuestionId  UserId  AnswerId  IsCorrect  CorrectAnswer  AnswerValue    [1307347 rows x 6 columns]

In [ ]:
data_relation = pd.merge(data_relation,data_ques)

In [ ]:
from re import sub


digit_list = []
for x in data_relation['SubjectId']:
    digit = re.findall("\d+",x)
    digit = [int(x) for x in digit]
    digit_list.append(digit)
# print(digit_list)
data_relation['SubjectId'] = digit_list
total_subject = []
for x in digit_list:
    total_subject += x
total_subject = set(total_subject)
# print(total_subject)
subject_redinx = {}
subject_num = len(total_subject)
total_subject = list(total_subject)
for i in range(subject_num):
    subject_redinx[total_subject[i]] = i
user2subject = {}


###stu_acc
grouped = data_relation.groupby('UserId')
for id,content in grouped:
    subjectOFnum = np.zeros(subject_num)
    subject_acc = np.zeros(subject_num)
    for x in content.index:
        subject = []
        for i in content['SubjectId'][x]:
            subject.append(subject_redinx[i])
        subject = np.array(subject)
        subjectOFnum[subject] += 1
        if content['IsCorrect'][x] == 1:
            subject_acc[subject] += 1
    subject_accTotal = np.zeros(subject_num)
    for i in range(subjectOFnum.shape[0]):
        if subjectOFnum[i] != 0:
            subject_accTotal[i] = subject_acc[i] / subjectOFnum[i]
    subject_expection = subject_accTotal * subjectOFnum
    user2subject[id] = subject_expection
for x,y in user2subject.items():
    print(x,y)

KeyboardInterrupt: 

In [ ]:
#total_acc


subject_total_num = np.zeros(subject_num)
subject_total_acc = np.zeros(subject_num)
for i in data_relation.index:
    subject_index =  [subject_redinx[x] for x in data_relation['SubjectId'][i]]
    subject_total_num[subject_index] += 1
    if int(data_relation['IsCorrect'][i]) == 1:
       subject_total_acc[subject_index] += 1
total_question_expection = subject_total_acc / subject_total_num * subject_total_num

In [ ]:
factor_list = []
for x in tqdm(range(data_stu['UserId'].shape[0])):
    factor_list.append(user2subject[data_relation['UserId'][x]] / total_question_expection)

100%|██████████| 1307347/1307347 [00:07<00:00, 174026.15it/s]

[array([1.59713201e-04, 1.66972783e-04, 2.59652585e-04, 0.00000000e+00,
       1.41732486e-04, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.24567707e-04, 2.38303891e-04, 1.28998968e-04, 0.00000000e+00,
       1.42166619e-04, 0.00000000e+00, 2.88278592e-04, 1.44341801e-04,
       2.21513379e-04, 2.61809478e-04, 9.71619009e-05, 0.00000000e+00,
       1.13274774e-04, 0.00000000e+00, 1.55351872e-04, 3.68239356e-04,
       3.78752012e-04, 0.00000000e+00, 7.61962816e-05, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.61848651e-04, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       1.48885838e-04, 1.28998968e-04, 1.44341801e-04, 2.21513379e-04,
       1.42166619e-04, 0.00000000e+00, 7.61962816e-05, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.06553010e-04, 1.68010753e-04,
       3.94321767e-04, 1.55351872e-04, 0.00000000e+00, 4.08496732e-04,
       3.93752461e-04, 0.00000000e+00, 0.00000000e+00, 1.19205495e-04,
     

In [ ]:
factor_list = np.array(factor_list)
svd = TruncatedSVD(4)
factor_list = svd.fit_transform(factor_list)
print(factor_list)

         QuestionId  UserId  AnswerId  IsCorrect  CorrectAnswer  AnswerValue  \
0               898    2111    280203          1              2            2   
1               898    4884   1355100          0              2            4   
2               898    3131    481614          0              2            4   
3               898    1921   1506133          1              2            2   
4               898    2471    498300          1              2            2   
...             ...     ...       ...        ...            ...          ...   
1307342         294    3042   1073567          1              3            3   
1307343         603    3042    503030          1              4            4   
1307344         603    5873   1156582          0              4            1   
1307345         603     811    468910          0              4            2   
1307346         603    5227    208288          0              4            2   

                SubjectId              

In [ ]:
factor_list_svd = pd.DataFrame(factor_list).add_prefix('factor_list_svd_')
factor_list_svd['UserId'] = data_stu['UserId']
data_stu = pd.merge(left=data_stu,right=factor_list_svd,on='UserId')
print(data_stu)